In [2]:
import scipy
import numpy
import scipy.io
import matplotlib.pyplot as plt
import glob
import json
import math
import os
import subprocess
import tempfile
import numpy as np
import regex

In [2]:
def run_ml(work_folder):
    stiffness=scipy.io.mmread(work_folder+"/stiffness.mat")
    solution=numpy.loadtxt(work_folder+"/solution.mat")
    rhs=(stiffness.tocsr()).dot(solution)
    rhs=numpy.reshape(rhs,(rhs.shape[0],1))
    scipy.io.mmwrite(work_folder+"/rhs.mtx",rhs)
    ml_exe="/home/yiwei/Trilinos/build/packages/ml/examples/BasicExamples/ML_read_MatrixMarket.exe"
    matrix=work_folder+"/stiffness.mat"
    rhs=work_folder+"/rhs.mtx"
    block_size=1
    solver="cg"
    output_folder=work_folder
    logname='/log_ml_'+str(block_size)+'.txt'
    if (not os.path.exists(output_folder)):
        os.makedirs(output_folder)
    logfile = open(output_folder+logname, "w")
    os.chdir(output_folder)
    args = [ml_exe,
    "--matrix="+matrix,"--krylov="+solver,"--npdes="+str(block_size),"--rhs="+rhs]
    subprocess.run(args,stdout=logfile,stderr=logfile) 
    logfile.close()
    return


In [3]:
def print_ml(path):
    logfile=open(path,'r')
    hierachy=[]
    rows=[]
    unknowns=[]
    iterations=-1
    for line in logfile.readlines():
        m=regex.match(r"Smoother \(level \d\) : # global rows = \d+",line)
        n=regex.search(r"total iterations: \d+",line)
        if m!=None:
            tmp=regex.findall(r"\d+",line)
            hierachy.append(int(tmp[0]))
            rows.append(int(tmp[1]))
            unknowns.append(int(tmp[2]))
        if n!=None:
            tmp=regex.findall(r"\d+",n.group(0))   
            iterations=int(tmp[0])
    for hie,unk,row in zip(hierachy,unknowns,rows):
        print("Level ",hie," Global rows ",row," Unknowns ", unk)
    print("Iterations",iterations)
        

In [16]:
# for folder in glob.glob("/home/yiwei/results_new/test_refine/3d_laplace/AMGCL_sa/**/**/**/**/**/**/**/output"):
#     print(folder)
#     run_ml(folder)

In [17]:
# for logfile in glob.glob("/home/yiwei/results_new/test_refine/3d_laplace/AMGCL_sa/**/**/**/**/**/**/**/output/log_ml_1.txt"):
#     print(logfile)
#     print_ml(logfile)

# Produce rhs

In [19]:
work_folder="/home/yiwei/results_new/test_refine_0.00/2d_bar_lowE/AMGCL/square_beam2d_structed/bar_2d/P1/ref0/block2/Thread32/0/output"
stiffness=scipy.io.mmread(work_folder+"/stiffness.mat")
solution=numpy.loadtxt(work_folder+"/solution.mat")
rhs=(stiffness.tocsr()).dot(solution)
rhs=numpy.reshape(rhs,(rhs.shape[0],1))
scipy.io.mmwrite(work_folder+"/rhs.mtx",rhs)

# Run ML

In [5]:
work_folder="/home/yiwei/results_new/test_refine_0.00/2d_bar_lowE/AMGCL/square_beam2d_structed/bar_2d/P1/ref0/block2/Thread32/0/output/new_matrix"
ml_exe="/home/yiwei/Trilinos/build/packages/ml/examples/BasicExamples/ML_read_MatrixMarket.exe"
matrix=work_folder+"/stiffness.mat"
rhs=work_folder+"/rhs.mtx"
block_size=2
solver="cg"
output_folder=work_folder
logname='/log_ml_'+str(block_size)+'.txt'
if (not os.path.exists(output_folder)):
    os.makedirs(output_folder)
logfile = open(output_folder+logname, "w")
os.chdir(output_folder)
args = [ml_exe,
"--matrix="+matrix,"--krylov="+solver,"--npdes="+str(block_size),"--rhs="+rhs]
subprocess.run(args,stdout=logfile,stderr=logfile) 
logfile.close()

# Print Hierachy

In [6]:
logfile=open(output_folder+logname, "r")
hierachy=[]
rows=[]
unknowns=[]
iterations=-1
for line in logfile.readlines():
    m=regex.match(r"Smoother \(level \d\) : # global rows = \d+",line)
    n=regex.search(r"total iterations: \d+",line)
    if m!=None:
        tmp=regex.findall(r"\d+",line)
        hierachy.append(int(tmp[0]))
        rows.append(int(tmp[1]))
        unknowns.append(int(tmp[2]))
    if n!=None:
        tmp=regex.findall(r"\d+",n.group(0))   
        iterations=int(tmp[0])
for hie,unk,row in zip(hierachy,unknowns,rows):
    print("Level ",hie," Global rows ",row," Unknowns ", unk)
print("Iterations",iterations)

Level  0  Global rows  120  Unknowns  1250
Level  1  Global rows  16  Unknowns  184
Iterations 74
